## PyTorch a Quick Start

- By default PyTorch has two primitives to work with data
    - The `torch.utils.data.Dataset` is used to store the samples and the corresponding labels.
    - The `torch.utils.data.DataLoader` is used to provide a highly custom wrapper around the dataset to aid with loading the data.

In [10]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

In [8]:
# Loading the Datasets
training_data = datasets.FashionMNIST(
    root="data", train=True, download=True, transform=ToTensor()
)
print("Training Data: ", training_data)

testing_data = datasets.FashionMNIST(
    root="data", train=False, download=True, transform=ToTensor()
)
print("Testing Data: ", testing_data)

100.0%
100.0%
100.0%
100.0%

Training Data:  Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()
Testing Data:  Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()


## DataLoaders
- DataLoaders support automatic batching, sampling, shuffling, and multiprocess data loading.
- They wrap the dataset over an iterable.

In [9]:
# Fixed Batch Size
batch_size = 64

# DataLoaders
train_loader = DataLoader(
    dataset=training_data, batch_size=batch_size
)
print("Training Loader: ", train_loader)

test_loader = DataLoader(
    dataset=testing_data, batch_size=batch_size
)
print("Testing Loader: ", test_loader)

Training Loader:  <torch.utils.data.dataloader.DataLoader object at 0x133006f00>
Testing Loader:  <torch.utils.data.dataloader.DataLoader object at 0x133005fd0>


## Building the Model
- Every neural network built using PyTorch inherits its properties for the `nn` module.
- All the layers of the model are defined inside the constructor of the class.
- The propagation of the data through the layers is defined in the forward function.

In [12]:
if torch.accelerator.is_available():
    acc = torch.accelerator.current_accelerator(check_available=True).type  # type: ignore
else:
    acc = "cpu"

print(f"Available Accelerator: {acc}")

Available Accelerator: mps


In [13]:
class FirstPyTorchNN(nn.Module):
    """This class implements a neural network in PyTorch."""
    def __init__(self) -> None:
        super().__init__()
        self.flatten = nn.Flatten()
        self.sequential = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.Softmax()
        )

    def forward(self, x):
        """Implements the forward propagation of the model."""
        x = self.flatten(x)
        logits = self.sequential(x)
        return logits

In [14]:
# Loading the Model on the Accelerator, Much more relevant in respect the workings of a GPU
first_model = FirstPyTorchNN().to(device=acc)
print(first_model)

FirstPyTorchNN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (sequential): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): Softmax(dim=None)
  )
)


In [15]:
# Other Hyper Parameters
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=first_model.parameters(), lr=1e-3)

In [22]:
# Training Method
def train(dataloader: DataLoader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(acc), y.to(acc)

        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss}  [{current}/{size}]")

for epoch in range(20):
    train(train_loader, first_model, loss_fn, optimizer)

/Users/raidenshogun/Development/PyTorch-Playgrounds/.venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1773: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


loss: 1.6955392360687256  [64/60000]
loss: 1.6377341747283936  [6464/60000]
loss: 1.7271332740783691  [12864/60000]
loss: 1.90253746509552  [19264/60000]
loss: 1.8375917673110962  [25664/60000]
loss: 1.75740647315979  [32064/60000]
loss: 1.7461334466934204  [38464/60000]
loss: 1.7571625709533691  [44864/60000]
loss: 1.7730157375335693  [51264/60000]
loss: 1.8529824018478394  [57664/60000]
loss: 1.6950182914733887  [64/60000]
loss: 1.6778686046600342  [6464/60000]
loss: 1.7261576652526855  [12864/60000]
loss: 1.833077311515808  [19264/60000]
loss: 1.7779645919799805  [25664/60000]
loss: 1.7593181133270264  [32064/60000]
loss: 1.7288976907730103  [38464/60000]
loss: 1.7537496089935303  [44864/60000]
loss: 1.8360141515731812  [51264/60000]
loss: 1.7953654527664185  [57664/60000]
loss: 1.6950538158416748  [64/60000]
loss: 1.6871020793914795  [6464/60000]
loss: 1.7565385103225708  [12864/60000]
loss: 1.8749682903289795  [19264/60000]
loss: 1.8169257640838623  [25664/60000]
loss: 1.854674696